In [ ]:
import os
from collections import defaultdict
import time

import numpy as np
import rf
import rf.imaging
import matplotlib.pyplot as plt
import scipy
from scipy import signal
from scipy.interpolate import interp1d
import obspy

from tqdm.auto import tqdm

In [ ]:
src_file = r"..\DATA\OA-ZRT-R-cleaned.h5"

In [ ]:
oa_all = rf.read_rf(src_file, "h5")

In [ ]:
# print(oa_all)

In [ ]:
def rf_to_dict(rf_data):
    db = defaultdict(lambda: defaultdict(list))
    for s in rf_data:
        _, sta, _, cha = s.id.split('.')
        db[sta][cha].append(s)
    return db

In [ ]:
def plot_station_rf_overlays(db_station):
    plt.figure(figsize=(16,24))
    colors = ["#8080a040", "#80a08040", "#a0808040"]
    min_x = 1e+20
    max_x = -1e20
    for i, (ch, streams) in enumerate(db_station.items()):
        if ch == 'size' or i >= 3:
            continue
        col = colors[i]
        plt.subplot(3, 1, i + 1)
        sta = streams[0].stats.station
        plt.title('.'.join([sta, ch]), fontsize=14)
        for j, s in enumerate(streams):
            lead_time = s.stats.onset - s.stats.starttime
            times = s.times()
            plt.plot(times - lead_time, s.data, '--', color=col, linewidth=2)
            if j == 0:
                data_mean = s.data
            else:
                data_mean += s.data
        data_mean /= float(j)
        plt.plot(s.times() - lead_time, data_mean, color="#202020", linewidth=2)
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude (normalized)')
        plt.grid(linestyle=':', color="#80808020")
        x_lims = plt.xlim()
        min_x = min(min_x, x_lims[0])
        max_x = max(max_x, x_lims[1])
    for i in range(3):
        subfig = plt.subplot(3, 1, i + 1)
        subfig.set_xlim((min_x, max_x))

In [ ]:
def compute_hk_stack(db_station, cha, h_range=np.linspace(20.0, 70.0, 501), k_range = np.linspace(1.4, 2.1, 351),
                     V_p = 6.4, sum_order=1, weighting=(0.5, 0.5)):
    # WARNING: If sum_order == 2, phase cancellation could be lost between streams,
    # since each stream's contribution will be made positive.
    
    # Pre-compute grid quantities
    k_grid, h_grid = np.meshgrid(k_range, h_range)
    hk_stack = np.zeros_like(k_grid)
    H_on_V_p = h_grid/V_p
    k2 = k_grid*k_grid

    include_t3 = (len(weighting) > 2) and (weighting[2] > 0)

    stream_stack = []
    cha_data = db_station[cha]
    # Loop over streams, compute times, and stack interpolated values at those times
    for s in cha_data:
        incidence = s.stats.inclination
        incidence_rad = incidence*np.pi/180.0
        cos_i, sin_i = np.cos(incidence_rad), np.sin(incidence_rad)
        sin2_i = sin_i*sin_i
        term1 = H_on_V_p*k_grid*np.abs(cos_i)
        term2 = H_on_V_p*np.sqrt(1 - k2*sin2_i)
        # Time for Ps
        t1 = term1 - term2
        # Time for PpPs
        t2 = term1 + term2
        if include_t3:
            # Time for PpSs + PsPs
            t3 = 2*term1
        interpolator = interp1d(s.times(), s.data, kind='cubic', copy=False, bounds_error=False, assume_sorted=True)
        if include_t3:
            stream_sum = weighting[0]*interpolator(t1) + weighting[1]*interpolator(t2) + weighting[2]*interpolator(t3)
        else:
            stream_sum = weighting[0]*interpolator(t1) + weighting[1]*interpolator(t2)

        if sum_order == 2:
            stream_sum = stream_sum*stream_sum  # Second order

        stream_stack.append(stream_sum)

    hk_stack = np.nanmean(np.array(stream_stack), axis=0)

    if sum_order == 2:
        hk_stack = np.sqrt(hk_stack)  # Second order
        
    return k_grid, h_grid, hk_stack

In [ ]:
def plot_hk_stack(k_grid, h_grid, hk_stack, title=None, save_file=None, sum_order=1, show=True, num=None):
    # Use a perceptually linear color map
    colmap = 'plasma'
    plt.figure(figsize=(16, 12))
    plt.contourf(k_grid, h_grid, hk_stack, levels=50, cmap=colmap)
    cb = plt.colorbar()
    if sum_order == 2:
        cb.ax.set_ylabel('Stack L2 norm')
    else:
        cb.ax.set_ylabel('Stack sum')
    plt.contour(k_grid, h_grid, hk_stack, levels=10, colors='k', linewidths=1)
    plt.xlabel(r'$\kappa = \frac{V_p}{V_s}$ (ratio)', fontsize=14)
    plt.ylabel('H = Moho depth (km)', fontsize=14)
    if title is not None:
        plt.title(title, fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    
    if num is not None:
        xl = plt.xlim()
        yl = plt.ylim()
        txt_x = xl[0] + 0.85*(xl[1] - xl[0])
        txt_y = yl[0] + 0.95*(yl[1] - yl[0])
        plt.text(txt_x, txt_y, "N = {}".format(num), color="#ffffff", fontsize=16, fontweight='bold')

    if save_file is not None:
        tries = 10
        while tries > 0:
            try:
                tries -= 1
                plt.savefig(save_file, dpi=300)
                break
            except PermissionError:
                time.sleep(1)
                if tries == 0:
                    print("WARNING: Failed to save file {} due to permissions!".format(save_file))
                    break
            # end try
        # end while

    if show:
        plt.show()
    else:
        plt.close()

# Select test station and plot overlaid stream data per channel

In [ ]:
db = rf_to_dict(oa_all)

In [ ]:
oa_bu24 = db['BU24']

In [ ]:
oa_bu24['HHR'][0].stats

In [ ]:
plot_station_rf_overlays(oa_bu24)

# Run over all channels to verify on HHR is the correct channel to plot

In [ ]:
# Plot stacks for all channels in BU24
weighting_bu24 = (0.6, 0.3, 0.1)
# for cha in ['HHR', 'HHT', 'HHZ']:
for cha in ['HHR']:
    k_grid, h_grid, hk_stack = compute_hk_stack(oa_bu24, cha, weighting=weighting_bu24)

    sta = oa_bu24[cha][0].stats.station

    num = len(oa_bu24[cha])
#     save_file = "BU24_{}_Vp=6.6.png".format(cha)
    save_file = None
    plot_hk_stack(k_grid, h_grid, hk_stack, title=sta + '.{}'.format(cha), num=num, save_file=save_file)

# Loop over all OA stations and plot HK-stacks

In [ ]:
cha = 'HHR'
pbar = tqdm(total=len(db))
show = False
weighting = (0.5, 0.5)
for sta, db_sta in db.items():
    pbar.set_description(sta)
    pbar.update()
    k_grid, h_grid, hk_stack = compute_hk_stack(db_sta, cha, weighting=weighting)
    sta = db_sta[cha][0].stats.station
    save_file = sta + "_{}_hk_stack.png".format(cha)
    num = len(db_sta[cha])
    plot_hk_stack(k_grid, h_grid, hk_stack, title=sta + '.{}'.format(cha), save_file=save_file,
                  show=show, num=num)
pbar.close()

## Plot histogram of group ID distribution

In [ ]:
groups = []
for sta, db_sta in db.items():
    ch_data = db_sta['HHR']
    for s in ch_data:
        groups.append(s.stats.rf_group)

In [ ]:
plt.hist(groups)
plt.show()

## Plot histogram of inclinations

In [ ]:
incs = []
for sta, db_sta in db.items():
    ch_data = db_sta['HHR']
    for s in ch_data:
        incs.append(s.stats.inclination)

In [ ]:
plt.hist(incs)
plt.show()